In [1]:
def set_paths():
    import os
    import sys

    USECASES_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "repositorios", "use-cases")
    if USECASES_SRC not in sys.path:
        sys.path.append(USECASES_SRC)

In [2]:
set_paths()

In [3]:
from churn_nrt.src.utils.spark_session import get_spark_session
sc, spark, sql_context = get_spark_session(app_name="profile_app_segmentation")

# Get variables for caracterization

In [4]:
from churn_nrt.src.projects.analysis.digital_scoring.app_segment_profile import get_segment_variables

In [5]:
df_app_segment=spark.read.load('/data/udf/vf_es/churn/digital/app_segments_freq_quant_20200801')

In [6]:
#Febrero

In [7]:
var_febrero=get_segment_variables(spark, df_app_segment, '20200229')

[Info] Loading different modules and selecting variables of interest for closing_day=20200229
[DataTemplate] __init__ | module name = customer_base | path = /data/udf/vf_es/churn_nrt/customer_base | verbose = True
[DataTemplate] get_module | module customer_base - for closing_day=20200229 save=True save_others=True force_gen=False
[DataTemplate] get_module | args: () | kwargs: {'add_tgs': True}
[CustomerBase] is_default_module | args: () | kwargs: {'add_tgs': True}
[CustomerBase] is_default_module | Module customer_base cannot be saved since level is different than nif_cliente or add_tgs is True
[DataTemplate] get_module | Not found a module - '/data/udf/vf_es/churn_nrt/customer_base/year=2020/month=2/day=29'. Starting generation...
[CustomerBase] build_module | closing_day=20200229 force_gen=False add_tgs=True level=nif_cliente
[CustomerBase] Get get_customer_base_segment base segment | date 20200229 save_others True level=NIF_CLIENTE
[CustomerBase] __get_customer_base date_=20200229 

In [10]:
var_febrero.repartition(300).write.save(
            '/data/udf/vf_es/churn/digital/var_profile_segments_freq_quant_20200229',
            format="parquet",
            mode="overwrite",
        )

Py4JJavaError: An error occurred while calling o2668.save.
: java.lang.OutOfMemoryError: GC overhead limit exceeded


In [ ]:
var_julio=get_segment_variables(spark, df_app_segment, '20200731')

In [ ]:
var_julio.repartition(300).write.save(
            '/data/udf/vf_es/churn/digital/var_profile_segments_freq_quant_20200731',
            format="parquet",
            mode="overwrite",
        )

## Caracterización simple: mean of basic variables

In [ ]:
'''
Import datasets with variables and segmentation column:

df=spark.read.load('/data/udf/vf_es/churn/digital/var_profile_segments_20200229')
df2=spark.read.load('/data/udf/vf_es/churn/digital/var_profile_segments_20200630')

'''

In [8]:
from churn_nrt.src.projects.analysis.digital_scoring.app_segment_profile import get_segment_profile

In [9]:
profile_febrero=get_segment_profile(var_febrero)

+-----------------+---------------------+
|      app_segment|avg_days_until_fin_bi|
+-----------------+---------------------+
|          no_user|   -1551.474717876467|
|       light_user|  -1371.3525983593656|
|       heavy_user|  -1071.9623028607477|
|short_period_user|  -1361.2639973593002|
|  occasional_user|  -1243.6935166994106|
+-----------------+---------------------+

[Info] Average days_until_f_fin_bi above
+-----------------+------------------+
|      app_segment|   avg_nb_rgus_nif|
+-----------------+------------------+
|          no_user| 4.239136918652966|
|       light_user| 4.339136702827636|
|       heavy_user| 4.154375222109838|
|short_period_user|4.1238071452982465|
|  occasional_user| 4.295084271307933|
+-----------------+------------------+

[Info] Average number of products above
+-----------------+-------------------------+
|      app_segment|avg_nb_fixed_services_nif|
+-----------------+-------------------------+
|          no_user|       0.8517839701104466|
|   

+-----------------+------------------+
|      app_segment|     gnv_total_mou|
+-----------------+------------------+
|          no_user| 147.0142264909297|
|       light_user|155.39164144206086|
|       heavy_user| 175.4753440614283|
|short_period_user|174.16558203422792|
|  occasional_user| 158.7893766812292|
+-----------------+------------------+

[Info] Average total minutes of calls above
+-----------------+--------------------------+
|      app_segment|avg_gnv_num_distinct_comps|
+-----------------+--------------------------+
|          no_user|         2.526739629841286|
|       light_user|         2.618170437959013|
|       heavy_user|         2.807304338317706|
|short_period_user|         2.793593417282263|
|  occasional_user|        2.6662469693952846|
+-----------------+--------------------------+

[Info] Average number of distinct competitors calls above
+-----------------+------------------+
|      app_segment|  avg_PRICE_TARIFF|
+-----------------+------------------+
|    

In [12]:
profile_febrero.columns

['app_segment',
 'avg_days_until_fin_bi',
 'avg_nb_rgus_nif',
 'avg_nb_fixed_services_nif',
 'avg_nb_fbb_services_nif',
 'avg_nb_mobile_services_nif',
 'avg_nb_tv_services_nif',
 'avg_social_networks',
 'avg_google_apps',
 'avg_internet',
 'avg_messaging',
 'avg_travel',
 'avg_music',
 'avg_e_commerce',
 'avg_technology',
 'avg_bank',
 'avg_news',
 'avg_streaming',
 'avg_sports',
 'avg_gnv_total_calls',
 'avg_gnv_perc_calls_competitors',
 'gnv_total_mou',
 'avg_gnv_num_distinct_comps',
 'avg_PRICE_TARIFF',
 'avg_PRICE_DATA',
 'avg_num_calls_w4',
 'avg_num_distinct_comps',
 'avg_sum_distinct_days_with_calls',
 'avg_num_distinct_operators',
 'avg_nif_port_freq_per_msisdn',
 'avg_total_acan',
 'avg_Bill_N1_Amount_To_Pay',
 'avg_billing_mean',
 'avg_num_tickets_tipo_tramitacion_w4',
 'avg_num_tickets_tipo_averia_w4',
 'avg_num_tickets_tipo_reclamacion_w4']

In [13]:
profile_febrero.select('app_segment',
 'avg_days_until_fin_bi',
 'avg_nb_rgus_nif',
 'avg_nb_fixed_services_nif',
 'avg_nb_fbb_services_nif',
 'avg_nb_mobile_services_nif',
 'avg_nb_tv_services_nif').toPandas()

,app_segment,avg_days_until_fin_bi,avg_nb_rgus_nif,avg_nb_fixed_services_nif,avg_nb_fbb_services_nif,avg_nb_mobile_services_nif,avg_nb_tv_services_nif
0,no_user,-1551.474718,4.239137,0.851784,0.823156,2.401141,0.544888
1,light_user,-1371.352598,4.339137,0.866858,0.841815,2.463728,0.539684
2,heavy_user,-1071.962303,4.154375,0.851010,0.824798,2.322144,0.515894
3,short_period_user,-1361.263997,4.123807,0.843976,0.817851,2.301204,0.502563
4,occasional_user,-1243.693517,4.295084,0.866050,0.844130,2.427711,0.532941


In [14]:
profile_febrero.select('app_segment',
 'avg_social_networks',
 'avg_google_apps',
 'avg_internet',
 'avg_messaging',
 'avg_travel',
 'avg_music').toPandas()

,app_segment,avg_social_networks,avg_google_apps,avg_internet,avg_messaging,avg_travel,avg_music
0,no_user,9.162737e+08,6.034702e+06,3.577183e+07,6.618329e+07,5.442641e+06,5.392240e+08
1,light_user,1.017126e+09,6.726314e+06,4.176992e+07,7.046263e+07,6.770024e+06,5.704101e+08
2,heavy_user,9.667255e+08,8.218883e+06,5.119416e+07,8.255504e+07,8.945764e+06,5.747125e+08
3,short_period_user,1.030809e+09,8.185102e+06,4.903541e+07,8.095190e+07,8.655832e+06,5.912660e+08
4,occasional_user,1.049481e+09,6.707661e+06,4.523206e+07,7.093972e+07,7.326615e+06,5.869446e+08


In [15]:
profile_febrero.select('app_segment',
'avg_e_commerce',
 'avg_technology',
 'avg_bank',
 'avg_news',
 'avg_streaming',
 'avg_sports').toPandas()

,app_segment,avg_e_commerce,avg_technology,avg_bank,avg_news,avg_streaming,avg_sports
0,no_user,7.041628e+06,1.465491e+07,3.269807e+06,1.513856e+07,2.016402e+08,4.017427e+06
1,light_user,8.398829e+06,1.856034e+07,3.631779e+06,1.658813e+07,2.271320e+08,4.558182e+06
2,heavy_user,1.192166e+07,2.225264e+07,5.578039e+06,2.381025e+07,2.662790e+08,6.245959e+06
3,short_period_user,1.058651e+07,2.180014e+07,4.414940e+06,1.930155e+07,2.682879e+08,5.057479e+06
4,occasional_user,1.006441e+07,2.021645e+07,3.761837e+06,1.701435e+07,2.389854e+08,4.806736e+06


In [16]:
profile_febrero.select('app_segment',
'avg_gnv_total_calls',
 'avg_gnv_perc_calls_competitors',
 'gnv_total_mou',
 'avg_gnv_num_distinct_comps').toPandas()

,app_segment,avg_gnv_total_calls,avg_gnv_perc_calls_competitors,gnv_total_mou,avg_gnv_num_distinct_comps
0,no_user,47.476726,0.467916,147.014226,2.526740
1,light_user,49.998973,0.470221,155.391641,2.618170
2,heavy_user,53.881408,0.494466,175.475344,2.807304
3,short_period_user,54.114402,0.495411,174.165582,2.793593
4,occasional_user,50.127812,0.480565,158.789377,2.666247


In [17]:
profile_febrero.select('app_segment',
'avg_PRICE_TARIFF',
 'avg_PRICE_DATA').toPandas()

,app_segment,avg_PRICE_TARIFF,avg_PRICE_DATA
0,no_user,27.654763,0.000879
1,light_user,27.953089,0.001156
2,heavy_user,28.023266,0.000939
3,short_period_user,28.008806,0.001240
4,occasional_user,27.882082,0.000928


In [18]:
profile_febrero.select('app_segment',
'avg_nif_port_freq_per_msisdn',
 'avg_total_acan').toPandas()

,app_segment,avg_nif_port_freq_per_msisdn,avg_total_acan
0,no_user,0.811501,0.764850
1,light_user,0.907080,0.947880
2,heavy_user,0.938906,1.015294
3,short_period_user,0.939075,0.991384
4,occasional_user,0.949794,1.009423


In [19]:
profile_febrero.select('app_segment',
'avg_Bill_N1_Amount_To_Pay',
 'avg_billing_mean').toPandas()

,app_segment,avg_Bill_N1_Amount_To_Pay,avg_billing_mean
0,no_user,78.283072,78.778838
1,light_user,79.933776,79.925804
2,heavy_user,74.878414,74.582256
3,short_period_user,74.769814,73.951980
4,occasional_user,78.113007,78.178244


In [20]:
profile_febrero.select('app_segment',
'avg_num_distinct_comps',
 'avg_sum_distinct_days_with_calls',
 'avg_num_distinct_operators').toPandas()

,app_segment,avg_num_distinct_comps,avg_sum_distinct_days_with_calls,avg_num_distinct_operators
0,no_user,0.025616,0.0,0.778688
1,light_user,0.029036,0.0,0.858550
2,heavy_user,0.045594,0.0,0.879699
3,short_period_user,0.040196,0.0,0.876259
4,occasional_user,0.033164,0.0,0.889850


In [21]:
 profile_febrero.select('app_segment',
'avg_num_calls_w4',
'avg_num_tickets_tipo_tramitacion_w4',
 'avg_num_tickets_tipo_averia_w4',
 'avg_num_tickets_tipo_reclamacion_w4').toPandas()

,app_segment,avg_num_calls_w4,avg_num_tickets_tipo_tramitacion_w4,avg_num_tickets_tipo_averia_w4,avg_num_tickets_tipo_reclamacion_w4
0,no_user,0.128301,0.038988,0.075728,0.011835
1,light_user,0.163423,0.047326,0.090644,0.014178
2,heavy_user,0.487630,0.074013,0.135360,0.022726
3,short_period_user,0.322163,0.052229,0.109276,0.018166
4,occasional_user,0.214180,0.058404,0.111128,0.016766


In [22]:
profile_julio=spark.read.load('/data/udf/vf_es/churn/digital/profile_julio_freq_quant')

In [23]:
profile_julio.select('app_segment',
 'avg_days_until_fin_bi',
 'avg_nb_rgus_nif',
 'avg_nb_fixed_services_nif',
 'avg_nb_fbb_services_nif',
 'avg_nb_mobile_services_nif',
 'avg_nb_tv_services_nif').toPandas()

,app_segment,avg_days_until_fin_bi,avg_nb_rgus_nif,avg_nb_fixed_services_nif,avg_nb_fbb_services_nif,avg_nb_mobile_services_nif,avg_nb_tv_services_nif
0,short_period_user,-1494.261770,4.142873,0.848176,0.822235,2.315071,0.521057
1,occasional_user,-1374.381413,4.303898,0.867579,0.846695,2.433965,0.542797
2,light_user,-1501.304166,4.350987,0.868815,0.844680,2.472477,0.550401
3,heavy_user,-1207.153933,4.169175,0.853875,0.828408,2.332856,0.531544
4,no_user,-1679.408458,4.250671,0.853858,0.826603,2.408167,0.552244


In [24]:
profile_julio.select('app_segment',
 'avg_social_networks',
 'avg_google_apps',
 'avg_internet',
 'avg_messaging',
 'avg_travel',
 'avg_music').toPandas()

,app_segment,avg_social_networks,avg_google_apps,avg_internet,avg_messaging,avg_travel,avg_music
0,short_period_user,2.265131e+09,1.228382e+07,8.655394e+07,1.457407e+08,1.792893e+07,1.278123e+09
1,occasional_user,2.349999e+09,1.233578e+07,8.071588e+07,1.314704e+08,1.828130e+07,1.249724e+09
2,light_user,2.180047e+09,1.056409e+07,7.299522e+07,1.253944e+08,1.525964e+07,1.140021e+09
3,heavy_user,2.199990e+09,1.244415e+07,9.092966e+07,1.489219e+08,1.956533e+07,1.279343e+09
4,no_user,1.854197e+09,8.185863e+06,5.946700e+07,1.136324e+08,1.132636e+07,9.760120e+08


In [25]:
profile_julio.select('app_segment',
'avg_e_commerce',
 'avg_technology',
 'avg_bank',
 'avg_news',
 'avg_streaming',
 'avg_sports').toPandas()

,app_segment,avg_e_commerce,avg_technology,avg_bank,avg_news,avg_streaming,avg_sports
0,short_period_user,2.027878e+07,7.613158e+07,5.504675e+06,1.534899e+07,6.297632e+08,5.437876e+06
1,occasional_user,1.922473e+07,7.204084e+07,4.866982e+06,1.480572e+07,5.353059e+08,5.541463e+06
2,light_user,1.599275e+07,6.235741e+07,4.388662e+06,1.269571e+07,4.835908e+08,4.465592e+06
3,heavy_user,2.214600e+07,8.219770e+07,6.118633e+06,1.695971e+07,6.063305e+08,6.350115e+06
4,no_user,1.183588e+07,4.771129e+07,3.568874e+06,1.015121e+07,3.768413e+08,3.526665e+06


In [26]:
profile_julio.select('app_segment',
'avg_gnv_total_calls',
 'avg_gnv_perc_calls_competitors',
 'gnv_total_mou',
 'avg_gnv_num_distinct_comps').toPandas()

,app_segment,avg_gnv_total_calls,avg_gnv_perc_calls_competitors,gnv_total_mou,avg_gnv_num_distinct_comps
0,short_period_user,64.803531,0.513538,229.370039,3.040761
1,occasional_user,60.185742,0.498427,211.655296,2.907038
2,light_user,60.152764,0.487592,206.747811,2.851894
3,heavy_user,64.036664,0.511635,230.593178,3.047704
4,no_user,57.016614,0.483663,195.993116,2.744111


In [27]:
profile_julio.select('app_segment',
'avg_PRICE_TARIFF',
 'avg_PRICE_DATA').toPandas()

,app_segment,avg_PRICE_TARIFF,avg_PRICE_DATA
0,short_period_user,28.229676,0.001045
1,occasional_user,27.853842,0.000928
2,light_user,27.985058,0.001067
3,heavy_user,28.129698,0.000759
4,no_user,27.666829,0.000879


In [28]:
profile_julio.select('app_segment',
'avg_nif_port_freq_per_msisdn',
 'avg_total_acan').toPandas()

,app_segment,avg_nif_port_freq_per_msisdn,avg_total_acan
0,short_period_user,0.969401,1.045118
1,occasional_user,0.964959,1.033668
2,light_user,0.923195,0.975112
3,heavy_user,0.961530,1.054688
4,no_user,0.821843,0.779621


In [29]:
profile_julio.select('app_segment',
'avg_Bill_N1_Amount_To_Pay',
 'avg_billing_mean').toPandas()

,app_segment,avg_Bill_N1_Amount_To_Pay,avg_billing_mean
0,short_period_user,76.047238,75.432292
1,occasional_user,79.115962,77.387635
2,light_user,80.979350,79.862770
3,heavy_user,76.040153,74.620440
4,no_user,79.630961,78.331195


In [30]:
profile_julio.select('app_segment',
'avg_num_distinct_comps',
 'avg_sum_distinct_days_with_calls',
 'avg_num_distinct_operators').toPandas()

,app_segment,avg_num_distinct_comps,avg_sum_distinct_days_with_calls,avg_num_distinct_operators
0,short_period_user,0.040353,0.0,0.903113
1,occasional_user,0.034839,0.0,0.903360
2,light_user,0.029695,0.0,0.873430
3,heavy_user,0.045410,0.0,0.899633
4,no_user,0.025024,0.0,0.788453


In [31]:
profile_julio.select('app_segment',
'avg_num_calls_w4',
'avg_num_tickets_tipo_tramitacion_w4',
 'avg_num_tickets_tipo_averia_w4',
 'avg_num_tickets_tipo_reclamacion_w4').toPandas()

,app_segment,avg_num_calls_w4,avg_num_tickets_tipo_tramitacion_w4,avg_num_tickets_tipo_averia_w4,avg_num_tickets_tipo_reclamacion_w4
0,short_period_user,0.360847,0.030659,0.105635,0.016693
1,occasional_user,0.253716,0.021879,0.097518,0.012005
2,light_user,0.195074,0.020404,0.085558,0.010558
3,heavy_user,0.507225,0.032933,0.120451,0.017102
4,no_user,0.150821,0.015382,0.067903,0.007632


In [ ]:
profile_febrero.repartition(300).write.save(
                '/data/udf/vf_es/churn/digital/profile_febrero_freq_quant', format="parquet", mode="overwrite")

## Caracterización compleja: get_model_pattern

Arbol de decisión que sea cada segmento versus el resto y ver las reglas de decisión que llevan a clasificar en un segmento

- Import dataset with segmentation column and variables:

In [4]:
df=spark.read.load('/data/udf/vf_es/churn/digital/var_profile_segments_freq_quant_20200731')

In [5]:
df = df.fillna(0)

In [6]:
df.groupby('app_segment').count().show()

+-----------------+------+
|      app_segment| count|
+-----------------+------+
|          no_user|439217|
|       light_user|779662|
|       heavy_user|548715|
|short_period_user|152156|
|  occasional_user|170758|
+-----------------+------+



El método asume que tr set es el training set scoreado por el modelo que se quiere perfilar; a partir de aquí, saco dos polos: top X% y bottom X% (argumento top ratio)

En nuestro caso, tenemos un conjunto con S + R filas (S muestras del segmento a caractrizar y R filas del resto)

**Preparación del tr set**

- Submuestrear el dataset original para que S=R, dando lugar a un set con el mismo número de muestras del segmento a caracterizar que del resto de segmentos

In [26]:
from pyspark.sql.functions import col, row_number, monotonically_increasing_id, row_number
import numpy as np
import pandas as pd
from pyspark.sql.window import Window

In [27]:
segment='no_user'

In [28]:
train_segment=df.filter(col('app_segment')==segment) #data for the segment of interest
count_segment=train_segment.count() #number of samples for that segment

train_other= df.filter(col('app_segment')!=segment) #data for the rest of the segments
count_other=train_other.count()

train_other_down=train_other.sample(withReplacement=False,fraction=float(count_segment)/float(count_other)) #data for the rest of segments downsampled (number of samples = count_segment)

train_final=train_segment.union(train_other_down) \
.withColumn("id", row_number().over(Window.orderBy(monotonically_increasing_id()))) #final dataset

count_final=train_final.count()

In [29]:
train_final.groupby('app_segment').count().show()

+-----------------+------+
|      app_segment| count|
+-----------------+------+
|          no_user|439217|
|       light_user|207241|
|       heavy_user|145923|
|short_period_user| 40147|
|  occasional_user| 45914|
+-----------------+------+



In [30]:
train_final.limit(10).toPandas()

,NUM_CLIENTE,nif_cliente,msisdn,app_segment,tgs_days_until_f_fin_bi,nb_rgus_nif,nb_fixed_services_nif,nb_fbb_services_nif,nb_mobile_services_nif,nb_tv_services_nif,...,sum_distinct_days_with_calls,num_distinct_operators,total_acan,nif_port_freq_per_msisdn,num_tickets_tipo_tramitacion_w4,num_tickets_tipo_averia_w4,num_tickets_tipo_reclamacion_w4,billing_mean,Bill_N1_Amount_To_Pay,id
0,424200180,24370227X,649944568,no_user,-105.0,1,0,0,1,0,...,0,1.0,0,1.0,0,0,0,0.000000,0.00,1
1,083787983,91551769G,610895809,no_user,239.0,4,1,1,2,1,...,0,1.0,0,1.0,0,0,0,91.118004,92.97,2
2,364406449,52529572M,671580167,no_user,53.0,4,1,1,2,0,...,0,1.0,0,1.0,0,0,0,57.142002,56.61,3
3,995483854,35149076R,648246859,no_user,100.0,6,2,2,2,1,...,0,2.0,2,1.5,1,1,0,0.000000,0.00,4
4,716982599,10168263B,692255119,no_user,356.0,5,1,1,3,1,...,0,0.0,0,0.0,0,0,0,59.405998,70.02,5
5,294286590,82281324P,696939558,no_user,69.0,4,1,1,2,1,...,0,0.0,0,2.0,0,0,0,43.374001,45.47,6
6,197419174,41123149B,686267622,no_user,434.0,3,1,1,1,0,...,0,2.0,1,2.0,0,0,0,54.054001,53.99,7
7,219620250,W4564719M,678119756,no_user,-15.0,4,1,1,2,1,...,0,0.0,0,0.0,0,0,0,161.038010,161.23,8
8,461253052,97846629P,699442832,no_user,-10000.0,1,0,0,1,0,...,0,1.0,0,1.0,0,0,0,30.990000,30.99,9
9,112796535,78445321P,658185197,no_user,39.0,4,1,1,2,1,...,0,0.0,0,0.0,0,0,0,41.931999,40.98,10


**Aplicar el método con top ratio = 50%** 

Así, le decimos que tome todas las muestras de ambas categorías

Habría que añadir una columna de score: el algoritmo la utiliza para ordenar y tomar los polos top y bottom; nos valdría con asignar un valor decreciente, de forma que, cuando se ordene por esta columna, las filas de S queden todas por encima de las filas de R. Ahora mismo, tenemos todas las columnas de S por encima de R

In [31]:
score_df=spark.createDataFrame(pd.DataFrame(np.linspace(1,0,count_final)))\
                    .withColumn("id", row_number().over(Window.orderBy(monotonically_increasing_id())))

In [32]:
score_df.count()

877673

In [33]:
count_final

877673

In [34]:
train_scored=train_final.join(score_df, on="id",how='left')

In [35]:
train_scored.filter(col('app_segment')==segment).count()

439217

In [36]:
439217*2

878434

In [37]:
train_scored=train_scored.orderBy('0',ascending=False).limit(878434)

In [38]:
train_scored.orderBy('0',ascending=False).limit(439218).toPandas()

,id,NUM_CLIENTE,nif_cliente,msisdn,app_segment,tgs_days_until_f_fin_bi,nb_rgus_nif,nb_fixed_services_nif,nb_fbb_services_nif,nb_mobile_services_nif,...,sum_distinct_days_with_calls,num_distinct_operators,total_acan,nif_port_freq_per_msisdn,num_tickets_tipo_tramitacion_w4,num_tickets_tipo_averia_w4,num_tickets_tipo_reclamacion_w4,billing_mean,Bill_N1_Amount_To_Pay,0
0,1,424200180,24370227X,649944568,no_user,-105.0,1,0,0,1,...,0,1.0,0,1.000000,0,0,0,0.000000,0.00,1.000000
1,2,083787983,91551769G,610895809,no_user,239.0,4,1,1,2,...,0,1.0,0,1.000000,0,0,0,91.118004,92.97,0.999999
2,3,364406449,52529572M,671580167,no_user,53.0,4,1,1,2,...,0,1.0,0,1.000000,0,0,0,57.142002,56.61,0.999998
3,4,995483854,35149076R,648246859,no_user,100.0,6,2,2,2,...,0,2.0,2,1.500000,1,1,0,0.000000,0.00,0.999997
4,5,716982599,10168263B,692255119,no_user,356.0,5,1,1,3,...,0,0.0,0,0.000000,0,0,0,59.405998,70.02,0.999995
5,6,294286590,82281324P,696939558,no_user,69.0,4,1,1,2,...,0,0.0,0,2.000000,0,0,0,43.374001,45.47,0.999994
6,7,197419174,41123149B,686267622,no_user,434.0,3,1,1,1,...,0,2.0,1,2.000000,0,0,0,54.054001,53.99,0.999993
7,8,219620250,W4564719M,678119756,no_user,-15.0,4,1,1,2,...,0,0.0,0,0.000000,0,0,0,161.038010,161.23,0.999992
8,9,461253052,97846629P,699442832,no_user,-10000.0,1,0,0,1,...,0,1.0,0,1.000000,0,0,0,30.990000,30.99,0.999991
9,10,112796535,78445321P,658185197,no_user,39.0,4,1,1,2,...,0,0.0,0,0.000000,0,0,0,41.931999,40.98,0.999990


In [39]:
train_scored.repartition(300).write.save(
            '/data/udf/vf_es/churn/digital/var_profile_segments_freq_quant_20200731_scored_no_user',
            format="parquet",
            mode="overwrite",
        )

In [40]:
excluded_cols=['id',
 'NUM_CLIENTE',
 'nif_cliente',
 'msisdn',
 'app_segment', '0']

In [41]:
variables=[col for col in train_scored.columns if col not in excluded_cols]

En cuanto a las variables importantes que se proporcionan como entrada, serían aquellas variables que utilizas para describir a cada cliente, de forma que el tr set quede con la siguiente estructura:variable1, variable2, ..., variableN, score_col

In [42]:
from churn_nrt.src.projects_utils.models.modeler_profiling import get_model_pattern

In [43]:
pattern=get_model_pattern(spark,train_scored,variables,method='dt',score_col='0',top_ratio=0.5,threshold=5)

[Info] Data filtered and labeled with success
[Info] Pandas Dataframe transformation size: 63192456 bytes
[Info] Data ready for explicability of the model
[Info] Selecting top nodes
[Info] Decision Tree method selected
Process successfully finished


In [44]:
pattern

["If (24.0539999008 < 'billing_mean' <= 34.8359985352) & ('num_calls_w4' <= 0.5) & ('social_networks' > 640648.0) & ('technology' <= 537060.0) & ('tgs_days_until_f_fin_bi' <= -345.5) & ('gnv_perc_calls_competitors' <= 0.989197522402) & (2100706.0 < 'streaming' <= 260246072.0) & ('e_commerce' <= 761005.5) & ('music' <= 48141724.0) & (51.9083328247 < 'gnv_total_mou' <= 626.158325195) & ('nb_rgus_nif' <= 4.5) & ('nb_mobile_services_nif' <= 1.5) & ('messaging' <= 91859060.0) & (10.6199998856 < 'PRICE_TARIFF' <= 31.9149999619) & ('num_tickets_tipo_averia_w4' <= 1.5)  then the observation is predicted as in group churn",
 "If (135.059997559 < 'Bill_N1_Amount_To_Pay' <= 161.01499939) & (140.507995605 < 'billing_mean' <= 150.729995728) & ('social_networks' <= 537888.5) & ('nif_port_freq_per_msisdn' <= 2.79999995232) & ('total_acan' <= 5.5) & ('internet' <= 798870.5) & (0.5 < 'nb_fixed_services_nif' <= 1.5) & ('gnv_perc_calls_competitors' > -0.5) & ('nb_mobile_services_nif' > 3.5) & ('e_commerc